## Datensatz 1 aufbereiten

In [2]:
import pandas as pd

# CSV Fuel_Consumption_2000-2022.csv laden 
df = pd.read_csv("../data/Fuel_Consumption_2000-2022.csv")

# Spalten bereinigen (Leerzeichen entfernen)
df.columns = df.columns.str.strip()

# Relevante Spalten auswählen
df = df[[
    'YEAR',
    'MAKE',
    'MODEL',
    'VEHICLE CLASS',
    'ENGINE SIZE',
    'CYLINDERS',
    'FUEL',
    'COMB (L/100 km)'  
]]

# Spalten vereinfachen (umbenennen)
df.columns = [
    'year',
    'brand',
    'model',
    'vehicle_class',
    'engine_size_l',
    'cylinders',
    'fuel_type',
    'consumption_l_per_100km'
]

# Kraftstoffcodes in lesbare Namen umwandeln
fuel_mapping = {
    "X": "Regular gasoline",
    "Z": "Premium gasoline",
    "D": "Diesel",
    "E": "Ethanol (E85)",
    "N": "Natural Gas"
}

df["fuel_type"] = df["fuel_type"].map(fuel_mapping)

# Zeilenanzahl ausgeben
print("Zeilenanzahl:", df.shape[0])
print(df.head())


Zeilenanzahl: 22556
   year  brand    model vehicle_class  engine_size_l  cylinders  \
0  2000  ACURA    1.6EL       COMPACT            1.6          4   
1  2000  ACURA    1.6EL       COMPACT            1.6          4   
2  2000  ACURA    3.2TL      MID-SIZE            3.2          6   
3  2000  ACURA    3.5RL      MID-SIZE            3.5          6   
4  2000  ACURA  INTEGRA    SUBCOMPACT            1.8          4   

          fuel_type  consumption_l_per_100km  
0  Regular gasoline                      8.1  
1  Regular gasoline                      7.6  
2  Premium gasoline                     10.0  
3  Premium gasoline                     11.5  
4  Regular gasoline                      8.6  


## Datensatz 2 aufbereiten

In [3]:


# Lade zweiten Datensatz fuel.csv
df2 = pd.read_csv("../data/fuel.csv")

# Spalten bereinigen
df2.columns = df2.columns.str.strip()

# Relevante Spalten auswählen und umbenennen
df2 = df2[[
    'year',
    'make',
    'model',
    'class',
    'engine_cylinders',
    'engine_displacement',
    'fuel_type',
    'combined_mpg_ft1'
]]

# Spalten vereinfachen
df2.columns = [
    'year',
    'brand',
    'model',
    'vehicle_class',
    'cylinders',
    'engine_size_l',
    'fuel_type',
    'mpg'
]

# mpg in Vebrauch in l/100km umrechnen
def mpg_to_l_per_100km(mpg):
    return 235.214583 / mpg

df2["consumption_l_per_100km"] = df2["mpg"].apply(mpg_to_l_per_100km)
df2 = df2.drop(columns=["mpg"])

# Spaltenreihenfolge anpassen fürs Zusammenführen
df2 = df2[[
    'year',
    'brand',
    'model',
    'vehicle_class',
    'engine_size_l',
    'cylinders',
    'fuel_type',
    'consumption_l_per_100km'
]]

print("Zeilenanzahl:", df2.shape[0])
print(df2.head())


Zeilenanzahl: 38113
   year       brand               model                vehicle_class  \
0  1984  Alfa Romeo           GT V6 2.5             Minicompact Cars   
1  1984  Alfa Romeo           GT V6 2.5             Minicompact Cars   
2  1984  Alfa Romeo  Spider Veloce 2000                  Two Seaters   
3  1984  Alfa Romeo  Spider Veloce 2000                  Two Seaters   
4  1984  AM General   DJ Po Vehicle 2WD  Special Purpose Vehicle 2WD   

   engine_size_l  cylinders fuel_type  consumption_l_per_100km  
0            2.5        6.0   Regular                11.760729  
1            2.5        6.0   Regular                11.760729  
2            2.0        4.0   Regular                11.200694  
3            2.0        4.0   Regular                11.200694  
4            2.5        4.0   Regular                13.836152  


C:\Users\lucac\AppData\Local\Temp\ipykernel_40520\1460603810.py:2: DtypeWarning: Columns (7,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../data/fuel.csv")


## Datensätze zusammenführen

In [4]:
# Merge = vertikales Zusammenführen
df_combined = pd.concat([df, df2], ignore_index=True)


# Vorschau
print("✅ Erfolgreich zusammengeführt. Zeilenanzahl:", df_combined.shape[0])
print(df_combined.head())

✅ Erfolgreich zusammengeführt. Zeilenanzahl: 60669
   year  brand    model vehicle_class  engine_size_l  cylinders  \
0  2000  ACURA    1.6EL       COMPACT            1.6        4.0   
1  2000  ACURA    1.6EL       COMPACT            1.6        4.0   
2  2000  ACURA    3.2TL      MID-SIZE            3.2        6.0   
3  2000  ACURA    3.5RL      MID-SIZE            3.5        6.0   
4  2000  ACURA  INTEGRA    SUBCOMPACT            1.8        4.0   

          fuel_type  consumption_l_per_100km  
0  Regular gasoline                      8.1  
1  Regular gasoline                      7.6  
2  Premium gasoline                     10.0  
3  Premium gasoline                     11.5  
4  Regular gasoline                      8.6  


## Vehicle_class generalisieren

In [5]:
# Vereinheitliche die Spalte: alles klein schreiben und Leerzeichen entfernen
df_combined["vehicle_class"] = df_combined["vehicle_class"].str.strip().str.lower()

# Mapping Dictionary (alles klein)
vehicle_class_map = {
    # Small cars
    "compact": "Car - Small",
    "subcompact": "Car - Small",
    "subcompact cars": "Car - Small",
    "minicompact": "Car - Small",
    "minicompact cars": "Car - Small",
    "compact cars": "Car - Small",
    "small station wagons": "Car - Small",

    # Midsize
    "mid-size": "Car - Midsize",
    "midsize cars": "Car - Midsize",
    "midsize station wagons": "Car - Midsize",
    "station wagon: small": "Car - Midsize",
    "station wagon: mid-size": "Car - Midsize",

    # Large
    "full-size": "Car - Large",
    "full-size cars": "Car - Large",
    "large cars": "Car - Large",
    "midsize-large station wagons": "Car - Large",

    # SUVs
    "suv": "SUV",
    "suv - small": "SUV",
    "suv - standard": "SUV",
    "suv: small": "SUV",
    "suv: standard": "SUV",
    "small sport utility vehicle 2wd": "SUV",
    "small sport utility vehicle 4wd": "SUV",
    "standard sport utility vehicle 2wd": "SUV",
    "standard sport utility vehicle 4wd": "SUV",
    "sport utility vehicle - 2wd": "SUV",
    "sport utility vehicle - 4wd": "SUV",

    # Pickup Trucks
    "pickup truck - small": "Pickup Truck",
    "pickup truck - standard": "Pickup Truck",
    "pickup truck: small": "Pickup Truck",
    "pickup truck: standard": "Pickup Truck",
    "small pickup trucks": "Pickup Truck",
    "small pickup trucks 2wd": "Pickup Truck",
    "small pickup trucks 4wd": "Pickup Truck",
    "standard pickup trucks": "Pickup Truck",
    "standard pickup trucks 2wd": "Pickup Truck",
    "standard pickup trucks 4wd": "Pickup Truck",
    "standard pickup trucks/2wd": "Pickup Truck",

    # Vans & Minivans
    "van": "Van/Minivan",
    "van - cargo": "Van/Minivan",
    "van - passenger": "Van/Minivan",
    "minivan": "Van/Minivan",
    "minivan - 2wd": "Van/Minivan",
    "minivan - 4wd": "Van/Minivan",
    "vans": "Van/Minivan",
    "vans, passenger type": "Van/Minivan",
    "vans, cargo type": "Van/Minivan",
    "vans passenger": "Van/Minivan",

    # Special purpose
    "two-seater": "Special",
    "two seaters": "Special",
    "special purpose vehicle": "Special",
    "special purpose vehicle 2wd": "Special",
    "special purpose vehicle 4wd": "Special",
    "special purpose vehicles": "Special",
    "special purpose vehicles/2wd": "Special",
    "special purpose vehicles/4wd": "Special"
}

# Mapping anwenden und fehlende Werte mit "Other" ersetzen
df_combined["vehicle_class"] = df_combined["vehicle_class"].map(vehicle_class_map).fillna("Other")

# Ausgabe zur Kontrolle
print("🔍 Vereinheitlichte vehicle_class Werte:", df_combined["vehicle_class"].unique())

# Ausgabe der Tabelle
print(df_combined)

# Vorschau nach dem Entfernen von Duplikaten
print("Zeilenanzahl:", df_combined.shape[0])



🔍 Vereinheitlichte vehicle_class Werte: ['Car - Small' 'Car - Midsize' 'Other' 'Special' 'Car - Large' 'SUV'
 'Van/Minivan' 'Pickup Truck']
       year  brand          model  vehicle_class  engine_size_l  cylinders  \
0      2000  ACURA          1.6EL    Car - Small            1.6        4.0   
1      2000  ACURA          1.6EL    Car - Small            1.6        4.0   
2      2000  ACURA          3.2TL  Car - Midsize            3.2        6.0   
3      2000  ACURA          3.5RL  Car - Midsize            3.5        6.0   
4      2000  ACURA        INTEGRA    Car - Small            1.8        4.0   
...     ...    ...            ...            ...            ...        ...   
60664  2017  Volvo       XC60 FWD            SUV            2.0        4.0   
60665  2017  Volvo       XC90 AWD            SUV            2.0        4.0   
60666  2017  Volvo       XC90 AWD            SUV            2.0        4.0   
60667  2017  Volvo  XC90 AWD PHEV            SUV            2.0        4.0   
60

## Fueltype generalisieren

In [6]:
# Zeige vorab alle Rohwerte zur Kontrolle
print("Verschiedene fuel_type Werte:", df_combined["fuel_type"].unique())

# Vereinheitliche fuel_type: alles klein, Leerzeichen entfernen
df_combined["fuel_type"] = df_combined["fuel_type"].str.lower().str.strip()

# Mapping-Dictionary
fuel_map = {
    "regular gasoline": "gasoline regular",
    "regular": "gasoline regular",
    "midgrade": "gasoline regular",
    "premium gasoline": "gasoline premium",
    "premium": "gasoline premium",
    "diesel": "diesel"
}

# Mapping direkt zurück in fuel_type schreiben
df_combined["fuel_type"] = df_combined["fuel_type"].map(fuel_map)

# Alle nicht gemappten Einträge (= alternative, hybrid, elektrisch etc.) entfernen
df_combined = df_combined[df_combined["fuel_type"].notna()]

# Kontrolle: Mögliche Werte nach Bereinigung
print("fuel_type nach Generalisierung:", df_combined["fuel_type"].unique())

# Vorschau
print(df_combined.head())

# Vorschau nach dem Entfernen von fuel_type alternative, hybrid, elektrisch etc.
print("Zeilenanzahl nach Bereinigung:", df_combined.shape[0])



Verschiedene fuel_type Werte: ['Regular gasoline' 'Premium gasoline' 'Ethanol (E85)' 'Natural Gas'
 'Diesel' 'Regular' 'Premium' 'CNG' 'Electricity'
 'Gasoline or natural gas' 'Gasoline or E85' 'Gasoline or propane'
 'Premium or E85' 'Premium Gas or Electricity' 'Midgrade'
 'Regular Gas and Electricity' 'Premium and Electricity'
 'Regular Gas or Electricity']
fuel_type nach Generalisierung: ['gasoline regular' 'gasoline premium' 'diesel']
   year  brand    model  vehicle_class  engine_size_l  cylinders  \
0  2000  ACURA    1.6EL    Car - Small            1.6        4.0   
1  2000  ACURA    1.6EL    Car - Small            1.6        4.0   
2  2000  ACURA    3.2TL  Car - Midsize            3.2        6.0   
3  2000  ACURA    3.5RL  Car - Midsize            3.5        6.0   
4  2000  ACURA  INTEGRA    Car - Small            1.8        4.0   

          fuel_type  consumption_l_per_100km  
0  gasoline regular                      8.1  
1  gasoline regular                      7.6  
2  gaso

## Brand und Model vereinheitlichen

In [7]:
# Vereinheitliche Brand und Model auf Großbuchstaben und entferne führende Leerzeichen
df_combined["brand"] = df_combined["brand"].str.upper().str.strip()
df_combined["model"] = df_combined["model"].str.upper().str.strip()




## Duplikate bereinigen

In [8]:

# Entferne nur komplett identische Zeilen
df_combined = df_combined.drop_duplicates()
# Vorschau nach dem Entfernen von Duplikaten
print("Zeilenanzahl nach Entfernen von Duplikaten:", df_combined.shape[0])




Zeilenanzahl nach Entfernen von Duplikaten: 52248


## Mittelwertbildung bei identischen Fahrzeugkonfigurationen

In [9]:
# Gruppiere nach eindeutiger Fahrzeugkonfiguration
df_combined = df_combined.groupby([
    'year',
    'brand',
    'model',
    'vehicle_class',
    'engine_size_l',
    'cylinders',
    'fuel_type'
], as_index=False).agg({
    'consumption_l_per_100km': 'mean'
})

# Vorschau nach dem Entfernen von Duplikaten
print("Zeilenanzahl nach Zusammenführung des Mittelwertes", df_combined.shape[0])


Zeilenanzahl nach Zusammenführung des Mittelwertes 34957


## Tabelle in neue CSV-Datei speichern

In [10]:
# neue csv-Datei speichern
df_combined.to_csv("../data/fuel_combined.csv", index=False)